In [1]:
##### 처음에만 사용해주세요 #####
# !pip install import_ipynb
# !pip install openpyxl==2.6.2
##### 처음에만 사용해주세요 #####

In [9]:
import import_ipynb
import kw_algorithm_third as kw
import price_agent
import openpyxl
import readAndWriteExcel as excel
import os

In [10]:
# 변수 세팅

# 백테스팅 가정치
ouput_keys_assumptions = ['매수대기일수', '최대보유일수', '익절 퍼센트', '손절 퍼센트']
buy_waiting_period_list = [5, 6] #공시 후 매수까지 avg_buy_price < target_date_low_price 되는지 기다리는 기간
holding_period_list = [60, 70] #매수 후 매도까지 홀딩하는 기간. 이 기간동안 목표 수익률을 넘게 될 경우 기간 전이라도 매도
selling_plus_percent_list = [0.10, 0.20] #홀딩기간 동안 익절 퍼센트
selling_minus_percent_list = [0.05, 0.10] #홀딩기간 동안 손절 퍼센트
# sheet_splitter_indices = [0]
# file_splitter_indices = [1, 2]

# output 키 리스트 작성
# 20210619 00:00:00 최종 업데이트
# 공시날짜, 종목코드, 종목명, 매수날짜, 수정 종가 반영 매수가격, 총 매수금액, 매도날짜, 수정 종가 반영 매도가격, 총 매도 금액, 수익률, 비고(익절, 손절, 홀딩 기간 도래 매도)
output_keys = {'eng': ['gonsi_date', 'gonsi_url', 'jongmok_name', 'jongmok_code', 'jongmok_market' ,'buy_data', 'buy_price', 'buy_qty', 'sell_price', 'sell_qty', 'yield', 'sell_type'],
               'kor': ['공시날짜', '공시url', '종목명', '종목코드', '마켓종류', '매수날짜', '매수가격', '매수금액', '매도날짜', '매도가격', '매도금액', '수익률', '매도유형']}
lang = 'kor'
jongmok_code_index = 3 # 걍 종목코드 키 위치
finance_base_date_index = 0 # 재무정보를 가져올 기준날짜 키 위치
monitoring_base_date_index = 0 # 120일간의 추세를 확인할 기준날짜 키 위치

# output 경로 입력
filepath_out = '.'
filename_out = 'backtestor'
sheetname_out_1 = 'Sheet1' # Sheet1 : 연도별 모든 재무데이터
sheetname_out_2 = 'Sheet2' # Sheet2 : 기준날짜 직전 재무데이터

# Finance 데이터 경로 입력
filepath_finance = 'financial'
filename_finance_sample = '000020' # 재무 키 리스트를 가져올 샘플 종목코드

# Price 데이터 경로 입력
filepath_price = '../kw/prices/all_daily_data_modified_close'
filepath_price_sample = '../kw/prices/all_daily_data'
filename_price_sample = '000020' # 영업일 리스트를 불러올 샘플 데이터

# 모니터링 기간 입력
monitoring_period = 120

In [11]:
# 테스트용 샘플값
sample = [['2018-01-02', '', '삼성SDI', '006400', '', '2018-01-03', 207000.0, 10000000, '2018-01-31', 196650.0, 9439200.0, -0.05608, '손절'],
         ['2018-01-02', '', '에스에너지', '095910', '', '2018-01-03', 7761.0, 10000000, '2018-01-10', 7372.95, 9496359.6, -0.05036, '손절'],
         ['2018-01-02', '', '엔케이맥스', '182400', '', '2018-01-03', 36659.907, 10000000, '2018-01-03', 34826.912, 9472920.064, -0.05271, '손절'],
         ['2018-01-02', '', '파커스', '065690', '', '2018-01-03', 3359.0, 10000000, '2018-01-09', 3191.05, 9499755.85, -0.05002, '손절'],
         ['2018-01-02', '', '피델릭스', '032580', '', '2018-01-03', 1974.0, 10000000, '2018-01-18', 1875.3, 9498394.5, -0.05016, '손절'],
         ['2018-01-02', '', '한국캐피탈', '023760', '', '2018-01-03', 566.961, 10000000, '2018-02-01', 623.657, 10999438.509000001, 0.09994, '익절'],
         ['2018-01-02', '', '휴니드테크놀러지스', '005870', '', '2018-01-03', 12367.126, 10000000, '2018-01-03', 11748.77, 9493006.16, -0.0507, '손절']]

In [12]:
# 기타 클래스/메소드 선언

# 걍 cartesian product 해주는거
def _cartProd2(a, b):
    prod = []
    for i in a:
        for j in b:
            try:
                k = [*i]
                k.append(j)
                prod.append(k)
            except:
                prod.append([i, j])
    return prod

def _cartProd(*argLists):
    prod = []
    for i, a in enumerate(argLists):
        if i > 0:
            prod = _cartProd2(prod, a)
        else:
            prod = a    
    return prod

set_assumptions = _cartProd(buy_waiting_period_list, holding_period_list, selling_plus_percent_list, selling_minus_percent_list)

# finance 업데이트용 클래스
class Finance:
    pivot_row = 6 # 엑셀 포맷이 바뀌면 수정 필요
    
    def __init__(self, filepath, sample, keys_assumptions, keys):
        self.filepath = filepath
        self.filename_sample = sample
        self.code_ws = {} # 중복으로 데이터를 불러오지 않기 위해 종목코드마다 불러온 워크시트를 할당. 
        # Sheet1 : 연도별 모든 재무데이터
        self.major_keys_1 = ['백테스팅 가정치'] + ['']*(len(keys_assumptions)-1) + ['기본정보'] + ['']*len(keys)
        self.minor_keys_1 = [*keys_assumptions]+[*keys]
        # Sheet2 : 기준날짜 직전 재무데이터
        self.major_keys_2 = [*self.major_keys_1]
        self.minor_keys_2 = [*self.minor_keys_1]
        self.error_list = []
        
        ws = excel.openfile(self.filepath, self.filename_sample)['Sheet1']
        self.code_ws[self.filename_sample] = ws
        self.year_cols = self._getYearCols(ws)
        self._appendKeys(ws)
        
    def updateAssumptions(self, bp, hp, sp, sm):
        self.assumptions = [bp, hp, sp, sm]
    
    def _appendKeys(self, ws):
        self.minor_keys_1.append(excel.cell(ws, 1, 4))
        self.minor_keys_2.append(excel.cell(ws, 1, 4))
        self.major_keys_2.append('직전년도 데이터')
        self.minor_keys_2.append('데이터 기준년도')
        
        for year in self.year_cols['list']:
            for row in range(self.pivot_row + 1, ws.max_row + 1):
                if row == self.pivot_row + 1:
                    self.major_keys_1.append('{}년 데이터'.format(str(year)))
                else:
                    self.major_keys_1.append('')
                self.minor_keys_1.append(excel.cell(ws, 1, row))
                
        for row in range(self.pivot_row + 1, ws.max_row + 1):
            self.major_keys_2.append('')
            self.minor_keys_2.append(excel.cell(ws, 1, row))
    
    # 모든 연도 목록과 해당 연도 열 위치 받아오기
    def _getYearCols(self, ws): 
        dict = {'list': []}
        for col in range(2, ws.max_column + 1):
            val = excel.cell(ws, col, self.pivot_row)
            if '/12' in val and not '/12(E)' in val:
                year = val.split('/')[0]
                dict['list'].append(int(year))
                dict[year] = col
        return dict

    # 주어진 종목코드에 해당하는 finance 데이터 추가
    def appendData(self, code, date, line):
        if not code in self.code_ws.keys():
            try:
                ws = excel.openfile(self.filepath, code)['Sheet1']
                self.code_ws[code] = ws
            except:
                self.error_list.append(code)
                return line, line
        else:
            ws = self.code_ws[code]
            
        line1 = self.assumptions + [*line]
        line2 = self.assumptions + [*line]
        line1.append(excel.cell(ws, 2, 4))
        line2.append(excel.cell(ws, 2, 4))
        
        for year in self.year_cols['list']:
            col = self.year_cols[str(year)]
            for row in range(self.pivot_row + 1, ws.max_row + 1):
                line1.append(excel.cell(ws, col, row))
        
        year_before = []
        year_now = int(date.split('-')[0])
        for year in self.year_cols['list']:
            if year < year_now:
                year_before.append(year)
        col = self.year_cols[str(max(year_before))]
        line2.append(max(year_before))
        for row in range(self.pivot_row + 1, ws.max_row + 1):
            line2.append(excel.cell(ws, col, row))
            
        return line1, line2
    
# 로그용 함수
def log(content):
    print('Log : {}'.format(content))

In [6]:
# 메인 코드

def run(for_test = False):
    wb, ws1, ws2 = excel.create(sheetname_out_1, sheetname_out_2)
    with_finance = Finance(filepath_finance, filename_finance_sample, ouput_keys_assumptions, output_keys[lang])
    keys_init = {'major1': with_finance.major_keys_1, 'major2': with_finance.major_keys_2, 'minor1': with_finance.minor_keys_1, 'minor2': with_finance.minor_keys_2}
    with_price = price_agent.Price(filepath_price, filepath_price_sample, filename_price_sample, monitoring_period, keys_init)
    excel.writeList(ws1, with_price.keys['major1'], is_first=True)
    excel.writeList(ws1, with_price.keys['minor1'])
    excel.writeList(ws2, with_price.keys['major2'], is_first=True)
    excel.writeList(ws2, with_price.keys['minor2'])
    
#     assert len(sheet_splitter_indices + file_splitter_indices) == len(set(sheet_splitter_indices + file_splitter_indices)), 'redundant index exists while splitting'

    for bp, hp, sp, sm in set_assumptions:
        with_finance.updateAssumptions(bp, hp, sp, sm)
        log('start backtesting with assumptions bp={}, hp={}, sp={}, sm={}'.format(bp, hp, sp, sm))
        if for_test: db_list = sample
        else: db_list = kw.run(bp, hp, sp, sm)
        print(db_list)
        log('the backtesting above ended')
        assert len(output_keys[lang]) == len(db_list[0]), 'different length between keys and values'
        for i, line in enumerate(db_list):
            line1_fi, line2_fi = with_finance.appendData(line[jongmok_code_index], line[finance_base_date_index], line)
            line1, line2 = with_price.appendData(line[jongmok_code_index], line[monitoring_base_date_index], with_price.nextDate(line[monitoring_base_date_index]), line1_fi, line2_fi)
            excel.writeList(ws1, line1)
            excel.writeList(ws2, line2)
        log('bactesting data successfully written')

    excel.save(filepath_out,filename_out, wb)
    log(with_finance.error_list)
    
if __name__ == '__main__':
    run(for_test = False)
    os.startfile('{}\{}.xlsx'.format(filepath_out.replace('/', 'W'), filename_out)) # 파일 열기 귀찮을때 쓰세욤

Log : excel sheet is created with title 'Sheet1'
Log : excel sheet is created with title 'Sheet2'
Log : open file '000020' at 'financial'
Log : start backtesting with assumptions bp=5, hp=60, sp=0.1, sm=0.05
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
종목 찾음! 108670
post된 avg_buy_price :  55700.000
2018-01-02 일 +  1 의 시가 96600
평균 매수 단가가 2018-01-02 일 +  1 일 시가보다 낮음!
2018-01-02 일 +  2 의 시가 97600
평균 매수 단가가 2018-01-02 일 +  2 일 시가보다 낮음!
2018-01-02 일 +  3 의 시가 95400
평균 매수 단가가 2018-01-02 일 +  3 일 시가보다 낮음!
2018-01-02 일 +  4 의 시가 95600
평균 매수 단가가 2018-01-02 일 +  4 일 시가보다 낮음!
2018-01-02 일 +  5 의 시가 94600
평균 매수 단가가 2018-01-02 일 +  5 일 시가보다 낮음!
종목 찾음! 017040
post된 avg_buy_price :  2468.227
2018-01-02 일 +  1 의 시가 2770
평균 매수 단가가 2018-01-02 일 +  1 일 시가보다 낮음!
2018-01-02 일 +  2 의 시가 2905
평균 매수 단가가 2018-01-02 일 +  2 일 시가보다 낮음!
2018-01-02 일 +  3 의 시가 2840
평균 매수 단가가 2018-01-02 일 +  3 일 시가보다 낮음!
2018-01-02 일 +  4 의 시가 2895
평균 매수 단가가 2018-01-02 일 +  4 일 시가보다 낮음!
2018-01-02 일 +  5 의 시가 2870
평균 매수 단가가 2018-

종목 찾음! 065690
post된 avg_buy_price :  3359.000
2018-01-02 일 +  1 의 시가 3270
평균 매수 단가가 2018-01-02 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 065690
김도형ㅄ6
low_check == False!! buyprice_modified =  3270
기준날짜 :  2018-01-02 매수 날짜 :  2018-01-03
손절 발생!!!!!!!!!!!!!!!!!
기준날짜 :  2018-01-02 매도 날짜 :  2018-01-11
저가 가격 :  3105
손절 대상 가격 :  3106.5
['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000355', '파커스', '065690', 'kosdaq', '2018-01-03', 3270, 10000000, '2018-01-11', 3106.5, 9500000.0, -0.05, '손절']
avg_buy_price가 0원이여서 제외
종목 찾음! 032580
post된 avg_buy_price :  1974.000
2018-01-02 일 +  1 의 시가 1960
평균 매수 단가가 2018-01-02 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 032580
김도형ㅄ6
low_check == False!! buyprice_modified =  1960
기준날짜 :  2018-01-02 매수 날짜 :  2018-01-03
손절 발생!!!!!!!!!!!!!!!!!
기준날짜 :  2018-01-02 매도 날짜 :  2018-01-23
저가 가격 :  1840
손절 대상 가격 :  1862.0
['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000038', '피델릭스', '032580', 'kosdaq', '2018-01-03', 1960, 1000000

기준날짜 :  2018-01-03 매도 날짜 :  2018-01-26
['2018-01-03', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180103000135', '디티알오토모티브', '007340', 'kospi', '2018-01-04', 31500, 10000000, '2018-01-26', 34650.0, 11000000.0, 0.1, '익절']
종목 찾음! 171120
post된 avg_buy_price :  14525.518
2018-01-03 일 +  1 의 시가 14150
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 171120
김도형ㅄ6
low_check == False!! buyprice_modified =  11541
기준날짜 :  2018-01-03 매수 날짜 :  2018-01-04
손절 발생!!!!!!!!!!!!!!!!!
기준날짜 :  2018-01-03 매도 날짜 :  2018-01-10
저가 가격 :  10847
손절 대상 가격 :  10963.95
['2018-01-03', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180103000286', '라이온켐텍', '171120', 'kosdaq', '2018-01-04', 11541, 10000000, '2018-01-10', 10963.95, 9500000.0, -0.05, '손절']
종목 찾음! 171120
post된 avg_buy_price :  14525.518
2018-01-03 일 +  1 의 시가 14150
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 171120
김도형ㅄ6
low_check == False!! buyprice_modified =  11541
기준날짜 :  2018-01-03 매수 날짜 :  2018-01-04
손절 발생!!!!!!!!

종목 찾음! 010780
post된 avg_buy_price :  35650.869
2018-01-03 일 +  1 의 시가 34500
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 010780
김도형ㅄ6
low_check == False!! buyprice_modified =  34500
기준날짜 :  2018-01-03 매수 날짜 :  2018-01-04
17 날짜의 익절 발생!!
기준날짜 :  2018-01-03 매도 날짜 :  2018-01-26
['2018-01-03', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180103000370', '아이에스동서', '010780', 'kospi', '2018-01-04', 34500, 10000000, '2018-01-26', 37950.0, 11000000.0, 0.1, '익절']
중복되었다!!!!
010780 2018-01-03 2018-01-26 종목코드, 공시날짜, selling 날짜
종목 찾음! 010780
post된 avg_buy_price :  35650.000
2018-01-03 일 +  1 의 시가 34500
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 010780
김도형ㅄ6
low_check == False!! buyprice_modified =  34500
기준날짜 :  2018-01-03 매수 날짜 :  2018-01-04
17 날짜의 익절 발생!!
기준날짜 :  2018-01-03 매도 날짜 :  2018-01-26
['2018-01-03', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180103000376', '아이에스동서', '010780', 'kospi', '2018-01-04', 34500, 10000000, '2018-01-26', 37950.0, 1100000

post된 avg_buy_price :  3740.000
2018-01-03 일 +  1 의 시가 3860
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 낮음!
2018-01-03 일 +  2 의 시가 3920
평균 매수 단가가 2018-01-03 일 +  2 일 시가보다 낮음!
2018-01-03 일 +  3 의 시가 4105
평균 매수 단가가 2018-01-03 일 +  3 일 시가보다 낮음!
2018-01-03 일 +  4 의 시가 4120
평균 매수 단가가 2018-01-03 일 +  4 일 시가보다 낮음!
2018-01-03 일 +  5 의 시가 4010
평균 매수 단가가 2018-01-03 일 +  5 일 시가보다 낮음!
종목 찾음! 003470
post된 avg_buy_price :  3820.000
2018-01-03 일 +  1 의 시가 3860
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 낮음!
2018-01-03 일 +  2 의 시가 3920
평균 매수 단가가 2018-01-03 일 +  2 일 시가보다 낮음!
2018-01-03 일 +  3 의 시가 4105
평균 매수 단가가 2018-01-03 일 +  3 일 시가보다 낮음!
2018-01-03 일 +  4 의 시가 4120
평균 매수 단가가 2018-01-03 일 +  4 일 시가보다 낮음!
2018-01-03 일 +  5 의 시가 4010
평균 매수 단가가 2018-01-03 일 +  5 일 시가보다 낮음!
종목 찾음! 003470
post된 avg_buy_price :  3735.000
2018-01-03 일 +  1 의 시가 3860
평균 매수 단가가 2018-01-03 일 +  1 일 시가보다 낮음!
2018-01-03 일 +  2 의 시가 3920
평균 매수 단가가 2018-01-03 일 +  2 일 시가보다 낮음!
2018-01-03 일 +  3 의 시가 4105
평균 매수 단가가 2018-01-03 일 +  3 일 시가보다 낮음!
2018-

...

post된 avg_buy_price :  7300.000
2018-01-05 일 +  1 의 시가 7190
평균 매수 단가가 2018-01-05 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 007330
김도형ㅄ6
low_check == False!! buyprice_modified =  7190
기준날짜 :  2018-01-05 매수 날짜 :  2018-01-08
16 날짜의 익절 발생!!
기준날짜 :  2018-01-05 매도 날짜 :  2018-01-29
['2018-01-05', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180105000401', '푸른저축은행', '007330', 'kosdaq', '2018-01-08', 7190, 10000000, '2018-01-29', 8628.0, 12000000.0, 0.2, '익절']
중복되었다!!!!
007330 2018-01-05 2018-01-29 종목코드, 공시날짜, selling 날짜
종목 찾음! 007330
post된 avg_buy_price :  7300.000
2018-01-05 일 +  1 의 시가 7190
평균 매수 단가가 2018-01-05 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 007330
김도형ㅄ6
low_check == False!! buyprice_modified =  7190
기준날짜 :  2018-01-05 매수 날짜 :  2018-01-08
16 날짜의 익절 발생!!
기준날짜 :  2018-01-05 매도 날짜 :  2018-01-29
['2018-01-05', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180105000407', '푸른저축은행', '007330', 'kosdaq', '2018-01-08', 7190, 10000000, '2018-01-29', 8628.0, 12000000.0, 0.2, '익절']
중복되었다!

손절 발생!!!!!!!!!!!!!!!!!
기준날짜 :  2018-01-08 매도 날짜 :  2018-03-22
저가 가격 :  28150
손절 대상 가격 :  28290.767
['2018-01-08', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180108000288', 'SK디앤디', '210980', 'kospi', '2018-01-09', 31434.186, 10000000, '2018-03-22', 28290.767, 8999999.872750005, -0.1, '손절']
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
avg_buy_price가 0원이여서 제외
종목 찾음! 023590
post된 avg_buy_price :  19323.099
2018-01-08 일 +  1 의 시가 23050
평균 매수 단가가 2018-01-08 일 +  1 일 시가보다 낮음!
2018-01-08 일 +  2 의 시가 23100
평균 매수 단가가 2018-01-08 일 +  2 일 시가보다 낮음!
2018-01-08 일 +  3 의 시가 22100
평균 매수 단가가 2018-01-08 일 +  3 일 시가보다 낮음!
2018-01-08 일 +  4 의 시가 20600
평균 매수 단가가 2018-01-08 일 +  4 일 시가보다 낮음!
2018-01-08 일 +  5 의 시가 211

종목 찾음! 014990
post된 avg_buy_price :  1155.000
2018-01-08 일 +  1 의 시가 1300
평균 매수 단가가 2018-01-08 일 +  1 일 시가보다 낮음!
2018-01-08 일 +  2 의 시가 1230
평균 매수 단가가 2018-01-08 일 +  2 일 시가보다 낮음!
2018-01-08 일 +  3 의 시가 1180
평균 매수 단가가 2018-01-08 일 +  3 일 시가보다 낮음!
2018-01-08 일 +  3 의 시가/저가 1180 / 1130
평균 매수 단가가 2018-01-08 일 +  3 일 시가보다 낮고, 저가보다는 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 014990
low_check == True!! buyprice_modified =  1155.0
참고용!! target_data_modified_low =  1130
기준날짜 :  2018-01-08 매수 날짜 :  2018-01-11
23 날짜의 익절 발생!!
기준날짜 :  2018-01-08 매도 날짜 :  2018-02-08
['2018-01-08', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180108000070', '인디에프', '014990', 'kospi', '2018-01-11', 1155.0, 10000000, '2018-02-08', 1386.0, 11999999.999999998, 0.2, '익절']
avg_buy_price가 0원이여서 제외
종목 찾음! 036420
post된 avg_buy_price :  5425.715
2018-01-08 일 +  1 의 시가 5190
평균 매수 단가가 2018-01-08 일 +  1 일 시가보다 높음!
매수!!!!
수정 종가 파일에서 종목 찾음! 036420
김도형ㅄ6
low_check == False!! buyprice_modified =  49352
기준날짜 :  2018-01-08 매수 날짜 :  2018-01-09
8 날

[['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000121', '대한방직', '001070', 'kospi', '2018-01-03', 15600, 10000000, '2018-01-23', 14040.0, 9000000.0, -0.1, '손절'], ['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000342', '디아이씨', '092200', 'kospi', '2018-01-03', 6965.633, 10000000, '2018-01-17', 8358.76, 12000000.574247884, 0.2, '익절'], ['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000219', '삼성SDI', '006400', 'kospi', '2018-01-03', 207000.0, 10000000, '2018-02-01', 186300.0, 9000000.0, -0.1, '손절'], ['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000398', '에스에너지', '095910', 'kosdaq', '2018-01-03', 7580, 10000000, '2018-02-06', 6822.0, 9000000.0, -0.1, '손절'], ['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000279', '엔케이맥스', '182400', 'kosdaq', '2018-01-03', 20250, 10000000, '2018-01-12', 24300.0, 12000000.0, 0.2, '익절'], ['2018-01-02', 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180102000

Log : bactesting data successfully written
Log : 'backtestor.xlsx' is saved at '.' folder
Log : []


In [13]:
import pickle

with open('../kw/prices/all_daily_data_modified_close/000020.pkl', 'rb') as f:
    df = pickle.load(f)

#     _modified_close

In [14]:
print(df)
idx = list(df.index).index('2016-05-17')
print(df[idx:idx+70])

             open   high    low  close  volume
2016-06-15  10350  10550  10250  10400  144971
2016-06-16  10400  11250  10300  10500  991321
2016-06-17  10750  11050  10000  10150  910307
2016-06-20  10300  10300  10000  10200  255159
2016-06-21  10250  10250   9990  10100  278627
...           ...    ...    ...    ...     ...
2021-04-26  15750  16350  15700  16250  645319
2021-04-27  16300  16500  16050  16100  355521
2021-04-28  16300  16300  15650  15800  415415
2021-04-29  15750  15800  15250  15350  249142
2021-04-30  15400  15600  15200  15350  160945

[1200 rows x 5 columns]


ValueError: '2016-05-17' is not in list

In [16]:
keys_init = {'major1': [], 'major2': [], 'minor1': [], 'minor2': []}
with_price = price_agent.Price(filepath_price, filepath_price, filename_price_sample, monitoring_period, keys_init)
print(with_price.nextDate('2018-01-02'))
print(with_price.appendData('001070', '2018-01-02', with_price.nextDate('2018-01-02'), [], []))

2018-01-03
Log : 001070.pkl is opened


KeyError: '05'

In [26]:
wb, ws1, ws2 = excel.create(sheetname_out_1, sheetname_out_2)
with_finance = Finance(filepath_finance, filename_finance_sample, ouput_keys_assumptions, output_keys[lang])
with_finance.updateAssumptions(0,0,0,0)
print(with_finance.appendData('277410', '2019-05-11', []))

Log : excel sheet is created with title 'Sheet1'
Log : excel sheet is created with title 'Sheet2'
Log : open file '000020' at 'financial'
Log : open file '277410' at 'financial'
([0, 0, 0, 0, '식품', '260', '45', '45', '41', '32', '32', '0', '290', '155', '134', '133', '1', '22', '44', '-44', '7', '20', '24', '113', '17.41', '12.42', '27.32', '12.21', '115.61', '530.36', '144', None, '601', None, '16', '0.72', '10.62', '22,177,049', '252', '42', '42', '-3', '-9', '-10', '0', '349', '107', '242', '241', None, '24', '28', '-21', '-18', '17', '11', '73', '16.71', '-3.69', '-5.09', '-2.91', '44.26', '878.68', '-42', 'N/A', '985', '1.61', '20', '1.26', '-51.37', '24,471,256', '259', '40', '40', '39', '32', '31', '0', '498', '190', '307', '306', None, '27', '38', '-63', '108', '57', '-19', '153', '15.44', '12.20', '11.45', '7.45', '61.89', '1,045.94', '120', '15.91', '1,156', '1.66', '30', '1.57', '25.36', '26,507,851'], [0, 0, 0, 0, '식품', 2018, '252', '42', '42', '-3', '-9', '-10', '0', '349'